# Load and display team info from JSON file

In [ ]:
import json

In [ ]:
with open('response_XXXXXXXXXXXXX.json') as f:
   team = json.load(f)
print(team)

In [ ]:
#access_key = (team['bucket_access_key'])
#secret_key = (team['bucket_access_secret'])
#print(access_key)
#print(secret_key)

# Download data from OpenSky S3 buckets

In [ ]:
# For Operating System commands
import os
# For high-level file operations
import shutil

import urllib3 # FIXME: only needed to disable InsecureRequestWarning

from pyopensky.config import opensky_config_dir
#
from traffic.core import Traffic

print(opensky_config_dir)

In [ ]:
from pyopensky.s3 import S3Client

In [ ]:
s3 = S3Client(cert_check=False) # FIXME: No cert checking: not nice
urllib3.disable_warnings() # FIXME: Not nice

# Local path to (capied) *.parquet files
strPqtLocPath = '.'
# Target pathfor filtered & resampled *.parquet files
strTgtPath = 'local_parquet_clean'

# Target sampling rate
strResample = '10s'

In [ ]:
allobj = []

In [ ]:
for obj in s3.s3client.list_objects("competition-data", recursive=True):
    print(f"{obj.bucket_name=}, {obj.object_name=}")
    allobj.append(obj)   

    # Assemble *.parquet path names
    strFile = obj.object_name
    strLocFullFile = os.path.join(strPqtLocPath, strFile)
    strTgtFullFile = os.path.join(strTgtPath, strFile)
    
    if os.path.isfile(strTgtFullFile):
        print('Skipping download, as already here: ' + strTgtFullFile)
    else:
        print('Download required ...')
        s3.download_object(obj)
        print(strLocFullFile)
  
        # Load *.parquet file into traffic object
        t = Traffic.from_file(strLocFullFile)
        # clean = filter + resample
        # t_clean = t.filter().resample(strResample).eval()
        # clean = filter + only first 20min + resample
        t_clean = t.first(minutes=20).filter().resample(strResample).eval()
        # Save clean traffic object as  *.parquet file
        print('Cleaned file will be saved here: ' + strTgtFullFile)
        t_clean.to_parquet(strTgtFullFile)
        # Delete temporary local file
        os.remove(strLocFullFile)

In [ ]:
len(allobj)